# SVMs (Support Vector Machines) con datos sintéticos

Basado en el capítulo 9 del libro "Introduction to Statistical Learning" de Gareth James, Daniela Witten, Trevor Hastie y Robert Tibshirani.

Cargamos las librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action=‘ignore’,category=DeprecationWarning)
#warnings.filterwarnings(action=‘ignore’,category=FutureWarning)

## 1. Creación de un dataset sintético aleatorio

Inicialmente, vamos a generar aleatoriamente un dataset sencillo de ensayo (20 instancias) para comprender como funciona el algoritmo SVM con un kernel lineal. El dataset va a tener dos variables numéricas predictoras (x1 y x2) y una clase (y)

In [ ]:
np.random.seed(5)
X = np.random.randn(20,2)
y = np.repeat([1,-1], 10)

# Trasladamos los puntos de la clase negativa un poco a la derecha, para incentivar la separabilidad lineal
X[y == -1] = X[y == -1] +1  

Creamos un dataframe con los datos anteriores

In [ ]:
df_0 = pd.concat([    pd.DataFrame(X), pd.DataFrame(y)   ], axis = 1)
df_0.columns = ['x1', 'x2', 'y']
df_0.head()

In [ ]:
pd.crosstab(index=df_0["y"], columns="count")

Vamos a ilustrar un particionamiento posible con los dataframes, usando el método **groupby**.
La idea es poder obtener subconjuntos de datos dados diferentes valores de una misma variable categórica.
En este caso, tenemos la varible objetivo *y*, que nos servirá como criterio de particionamiento.

El método *groupby* retorna una tupla con la categoría de la variable particionada (-1 o +1), y un dataframe con los registros correspondientes a cada categoría, incluyendo el índice (de la fila original) de los registros correspondientes.

In [ ]:
for name, group in df_0.groupby('y'):
    print("Grupo para la clase {}\n{}".format(name, group))

Vamos entonces a visualizar los datos en un scatterplot para poder entenderlos mejor. Vamos a utilizar un scatterplot y un gráfico de densidad de 2 dimensiones.

In [ ]:
fig, axs = plt.subplots(figsize=[15, 5], ncols=2, nrows=1)

for name, group in df_0.groupby('y'):
    sns.scatterplot(x="x1", y="x2", data=group, alpha=0.5, label=name, ax=axs[0])    
axs[0].legend()
axs[0].set_title("ScatterPlot: x1 vs. x2")

for name, group in df_0.groupby('y'):
    sns.kdeplot(data=group["x1"], data2=group["x2"], shade=True, shade_lowest=False, alpha=0.5,
                label=name, ax=axs[1])    
axs[1].legend(loc=2)
axs[1].set_title("Densidad de puntos: x1 vs. x2")

## 2. Entrenamiento de un modelo SVM de Clasificación

En Python, los modelos SVM de clasificación son representados por la clase **SVC**. En el caso de clasificación con más de 2 clases, **SVC** adopta una aproximación "uno contra uno" para determinar la clase final.

Esta clase se crea con los siguientes parámetros:

- *C*: valor de la penalidad de una instancia por violación de las márgenes (1 por defecto); controla la regularización. A menor valor de C, mayor amplitud de la margen del clasificador
- *kernel*: el tipo de kernel a utilizar ('rbf' por defecto); otros valores aceptados son: ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’. Se puede definir un kernel implementado en una clase por fuera del API. 
- *degree*: grado del polinomio en el caso del kernel 'poly' (3 por defecto); se ignora para otros kernels.
- *gamma*: determina la dispersión alrededor de los vectores de soporte ('auto' por defecto); solo se considera para los kernels ‘rbf’, ‘poly’ y ‘sigmoid’. A mayor *gamma*, mayor complejidad (dimensionalidad) del espacio de representación de las instancias.
- *probability* : Indica si se deben generar estimados de probabilidad de clase (False por defecto), lo que implicaría un costo en el tiempo de procesamiento.

Intentamos la clasificación con un kernel **lineal**, usando entonces un **clasificador de margen suave**. Como los datos no son linealmente separables, es necesario establecer un valor de C > 0.

In [ ]:
svc = SVC(C=1, kernel='linear')

A manera de ilustración, vamos a entrenarlo con todo el dataset (uds ya saben que esto no se debe hacer!!! Pero vamos a ir aumentando la complejidad del proceso poco a poco).

In [ ]:
svc.fit(X, y)

En el modelo entrenado podemos tener acceso a los vectores de soporte

In [ ]:
print("De los {} puntos de entrenamiento, {} son vectores de soporte".format(len(y), len(svc.support_)))

Vamos a definir una función **plot_svc** (tomado de R. Jordan Crouser @ Smith College) que nos permitirá visualizar los resultados de la aplicación de un modelo de SVM para la clasificación de un conjunto de datos. Recibe:

- svc: El modelo entrenado de tipo SVC
- X: La matriz con las variables predictoras X, del dataset a evaluar
- y: El array con las clases verdaderas correspondientes, del dataset a evaluar
- pad: El tamaño del borde a dejar sin puntos, valor por defecto 0.25
- h: la distancia entre los puntos en la grilla a evaluar, valor por defecto 0.02

In [ ]:
def plot_svc(svc, X, y, h=0.02, pad=0.25):
    # Determinamos los límites del plot a gráficar
    x1_min, x1_max = X[:, 0].min()-pad, X[:, 0].max()+pad
    x2_min, x2_max = X[:, 1].min()-pad, X[:, 1].max()+pad

    # Creamos una grilla de puntos que se van a enviar al clasificador (meshgrid)
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, h), np.arange(x2_min, x2_max, h))
    
    # Usando unravel(), vamos aplanar las matrices xx1 y xx2, creando una gran lista de puntos que se enviarán al modelo
    Z = svc.predict(np.c_[xx1.ravel(), xx2.ravel()]) #En Z quedan las clases predichas para cada punto de la grilla

    # Ploteamos la grilla con sus clases
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, cmap=plt.cm.Paired, alpha=0.2)

    # Ploteamos los puntos evaluados con sus clases reales
    plt.scatter(X[:, 0], X[:, 1], s=70, c=y, cmap=plt.cm.Paired)

    # Identificamos los vectores de soporte del modelo con una x
    sv = svc.support_vectors_
    plt.scatter(sv[:,0], sv[:,1], c='k', marker='x', s=100, linewidths=1)

    plt.xlim(x1_min, x1_max)
    plt.ylim(x2_min, x2_max)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()    

Veamos como nos va con todo el dataset:

In [ ]:
plot_svc(svc, X, y)

Vemos el espacio de representación bidimensional de los datos dividido en 2 por una frontera de decisión dada por el clasificador. Dado el kernel utilizado por el modelo, la frontera es lineal.

Encontramos unos cuantos errores, y podemos ver los puntos que intervienen como **vectores de soporte**, graficados con una "x". Podemos ver que estos puntos están cerca de la frontera de decisión. Algunos de ellos están en el lado de decisión incorrecto, no solo violan la margen, sino que quedan mal clasificados. Otros puntos están en el buen lado, pero violan la margen establecida por el clasificador (la margen no es visible).

Podemos identificar los vectores de soporte con las variables **support_**, que indica los índices de los puntos, y **support_vectors_**, que contiene su representación en el espacio de las variables independientes:

In [ ]:
svc.support_

In [ ]:
svc.support_vectors_

El parámetro *C* del modelo controla el valor de cada violación de la margen. 
Se trata de un parámetro de regularización que permite encontrar un balance entre las contribuciones del error de sesgo y de varianza.

Entre más pequeño sea de *C*, mayor será el margen y mas puntos quedarán identificados como vectores de soporte (tendiendo hacia el **underfitting**). 
Al contrario, entre más grande sea *C*, menor el margen y menor la cantidad de vectores de soporte (tendiendo hacia el **overfitting**). 

A continuación vamos a crear otro modelo SVM con otro valor de C, para entender mejor su influencia en los resultados obtenidos.

In [ ]:
svc = SVC(C=0.01, kernel='linear')
svc.fit(X, y)
print("De los {} puntos de entrenamiento, {} son vectores de soporte".format(len(y), len(svc.support_)))
plot_svc(svc, X, y)

In [ ]:
svc = SVC(C=0.1, kernel='linear')
svc.fit(X, y)
print("De los {} puntos de entrenamiento, {} son vectores de soporte".format(len(y), len(svc.support_)))
plot_svc(svc, X, y)

Vamos ahora a utilizar la clase **GridSearchCV** para aplicar el protocolo de evaluación de cross-validation y así poder estimar la mejor combinación de valores de los parámetros configurables de un modelo. Variaremos *C*, utilizando *accuracy* como métrica de evaluación.

En total se evaluarán 7 valores de cada uno de los dos parámetros, por lo que ensayaremos se crearán 7 modelos con configuraciones diferentes, cada uno con varias repeticiones de cross-validation.

In [ ]:
parametros = [{'C': [0.001, 0.01, 0.1, 1, 5, 10, 100]}]
clf = GridSearchCV(SVC(kernel='linear'), # el modelo a configurar
                   parametros, # los parámetros con los valores a ensayar
                   cv=10, # para cada modelo se hacen 10 repeticiones
                   scoring='accuracy', # métrica de evaluación a considerar para decidir el mejor modelo
                   return_train_score=True) # guardar en memoria los valores de las métricas de evaluación
clf.fit(X, y)

La búsqueda por grilla graba la configuración y los resultados de la evaluación del mejor modelo:

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_index_

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

Podemos acceder a los resultados de las evaluaciones, utilizando el diccionario que queda en la variable cv_results_.
Se puede consultar en su totalidad:

In [ ]:
clf.cv_results_

In [ ]:
clf.cv_results_.keys()

También se puede consultar elemento por elemento, dado que es un diccionario:

In [ ]:
clf.cv_results_['mean_train_score']

## 3. Predicción con el modelo entrenado

Creamos un SVC con kernel lineal y la mejor configuración encontrada (C=0.001) y lo usamos para predecir las clases, utilizando el método **predict** sobre un nuevo dataset de test que siga el mismo proceso generador del dataset de entrenamiento.

In [ ]:
np.random.seed(1)
X_test = np.random.randn(20,2)
y_test = np.random.choice([-1,1], 20)
X_test[y_test == 1] = X_test[y_test == 1] -1

In [ ]:
svc2 = SVC(C=0.1, kernel='linear', probability=True)
svc2.fit(X, y)
y_pred = svc2.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred), index=svc2.classes_, columns=svc2.classes_)

Tenemos 14 instancias bien clasificadas.

Podemos obtener el valor de decisión que permite eventualmente ver que tan lejos se está o no de las márgenes, y en que lado:

In [ ]:
svc2.decision_function(X_test)

Aquí podemos ver que solo hay dos puntos clasificados en la clase negativa, y la magnitud de este valor nos indica el nivel de confianza del clasificador, tal como lo indican las probabilidades de clase que se obtienen con la función **predict_proba** (solo disponible si se ha creado el objeto **SVC** con probability=True.

In [ ]:
svc2.predict_proba(X_test)

## 4. SVMs con un dataset linealmente separable

Modificamos el dataset de test anterior para que sea apenas linealmente separable (muy justo), y analizamos los resultados de varias configuraciones de un SVM aprendido sobre él

In [ ]:
X_test[y_test == 1] = X_test[y_test == 1] -1

In [ ]:
plt.scatter(X_test[:,0], X_test[:,1], s=70, c=y_test, cmap=plt.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')

Entrenemos un modelo (utilizando el test set como set de aprendizaje, solo como ilustración) que garantice que todas las instancias están bien clasificadas, acercándose a lo que sería un **clasificador de margen dura**. Para tal motivo, vamos a especificar un gran valor del parámetro *C* (10000), para que cada error tenga un gran valor y se busque la perfección de la clasificación del set de entrenamiento.

In [ ]:
svc3 = SVC(C=1e5, kernel='linear')
svc3.fit(X_test, y_test)
plot_svc(svc3, X_test, y_test)

Vemos que en efecto que no hubo errores en el entrenamiento, y que solo tenemos 3 vectores de soporte.
Como hay muchos puntos cercanos a la frontera de decisión que no son vectores de soporte, podemos asimilar que la margen es bastante angosta.
Podríamos estimar que este modelo es demasiado rigido y no tiene una buena capacidad de generalización (overfitting).

Intentemos con un valor de C=1.

In [ ]:
svc4 = SVC(C=1, kernel='linear')
svc4.fit(X_test, y_test)
plot_svc(svc4, X_test, y_test)

Aunque hay un error, vemos que hay una mayor cantidad de vectores de soporte alejados de la frontera de decisión, por lo que este modelo puede ser mas robusto frente al overfitting.

## 5. Creación de un dataset sintético no linealmente separable

Ahora, vamos a generar aleatoriamente un dataset sencillo de ensayo (300 instancias) para comprender como funciona el algoritmo, que va a tener dos variables numéricas predictoras (x1 y x2) y una clase (y):

- Las primeras 100 instancias van a tener valores pequeños en las dos variables, y se asignarán a la clase -1
- Las 100 instancias intermedias van a tener valores medianos en las dos variables, y se asignarán a la clase +1
- Las últimas 100 instancias van a tener valores grandes en las dos variables, y se asignarán a la clase -1

In [ ]:
np.random.seed(12345)
X = np.random.randn(300,2)
X[:100] = X[:100] -2.5
X[201:] = X[201:] +2.5
y = np.concatenate([np.repeat(-1, 100), np.repeat(1,100), np.repeat(-1, 100)])

Creamos un dataframe con los datos anteriores

In [ ]:
df_0 = pd.concat([    pd.DataFrame(X), pd.DataFrame(y)   ], axis = 1)
df_0.columns = ['x1', 'x2', 'y']

Agrupamos por clases y visualizamos los datos

In [ ]:
fig, axs = plt.subplots(figsize=[15, 5], ncols=2, nrows=1)

for name, group in df_0.groupby('y'):
    sns.scatterplot(x="x1", y="x2", data=group, alpha=0.5, label=name, ax=axs[0])    
axs[0].legend()
axs[0].set_title("ScatterPlot: x1 vs. x2")

for name, group in df_0.groupby('y'):
    sns.kdeplot(data=group["x1"], data2=group["x2"], shade=True, shade_lowest=False, alpha=0.5,
                label=name, ax=axs[1])    
axs[1].legend(loc=2)
axs[1].set_title("Densidad de puntos: x1 vs. x2")

### Entrenamiento y búsqueda de parámetros para un modelo SVM de Clasificación

Vamos a generar un conjunto de datos de entrenamiento y un set de evaluación utilizando la función **train_test_split**, dejando el 70% de los datos en el primer conjunto.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=666)

Vamos a crear un clasificador SVM con la clase **SVC**, a entrenarlo y a visualizar su frontera de clasificación y su error de test:

In [ ]:
svc3 = SVC(C=10, kernel='rbf', gamma=1)
svc3.fit(X_train, y_train)
print("De los {} puntos de entrenamiento, {} son vectores de soporte".format(len(y_train), len(svc3.support_)))
plot_svc(svc3, X_train, y_train)

In [ ]:
plot_svc(svc3, X_test, y_test)

In [ ]:
y_pred = svc3.predict(X_train)
pd.DataFrame(confusion_matrix(y_train, y_pred), index=svc3.classes_, columns=svc3.classes_)

In [ ]:
y_pred = svc3.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred), index=svc3.classes_, columns=svc3.classes_)

Con C=10, encontramos 7 errores de entrenamiento y 9 errores de test de 90 (90% de accuracy).

Veamos que pasa si incrementamos el valor de la sensibilidad a los errores a 100, haciendo que la margen sea más angosta.

In [ ]:
# Increasing C parameter, allowing more flexibility
svc4 = SVC(C=100, kernel='rbf', gamma=1.0)
svc4.fit(X_train, y_train)
print("De los {} puntos de entrenamiento, {} son vectores de soporte".format(len(y_train), len(svc4.support_)))
plot_svc(svc4, X_train, y_train)

Vemos que empieza a presentar overfitting, con una frontera de decisión más compleja que incluye "islas" para evitar errores. 
En cuanto a la calidad de las predicciones tenemos:

In [ ]:
y_pred = svc4.predict(X_train)
pd.DataFrame(confusion_matrix(y_train, y_pred), index=svc4.classes_, columns=svc4.classes_)

In [ ]:
y_pred = svc4.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred), index=svc3.classes_, columns=svc3.classes_)

Con C=100, encontramos 2 errores de entrenamiento y 53 errores de test de 90 (41.1% de accuracy).

Ahora, hagamos variar el parámetro **gamma**, del kernel **rbf**, que controla la amplitud de la gaussiana. Teniendo en cuenta que el valor original era 1, analicemos el efecto de utilizar valores más pequeños (0.1) y más grandes (5).

In [ ]:
svc5 = SVC(C=10, kernel='rbf', gamma=0.1)
svc5.fit(X_train, y_train)
print("De los {} puntos de entrenamiento, {} son vectores de soporte".format(len(y_train), len(svc5.support_)))
plot_svc(svc5, X_train, y_train)

Encontramos que a menor **gamma**, mas sencillo el modelo,  menor número de vectores de soporte y se tiende hacia el underfitting.

In [ ]:
svc6 = SVC(C=10, kernel='rbf', gamma=5)
svc6.fit(X_train, y_train)
print("De los {} puntos de entrenamiento, {} son vectores de soporte".format(len(y_train), len(svc6.support_)))
plot_svc(svc6, X_train, y_train)

Encontramos que entre mayor el **gamma**, mas complejo el modelo por lo que más vectores de soporte, y mas overfitting.

Utilicemos ahora **GridSearchCV** para estimar la mejor combinación de valores de los parámetros configurables (en este caso variaremos *C* y *gamma*), con respecto a una métrica de evaluación (en este caso utilizaremos el *accuracy*).
En total se evaluarán 5 valores de cada uno de los dos parámetros, por lo que ensayaremos con 25 configuraciones diferentes.

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = [{'C': [0.01, 0.1, 1, 10, 100],
                     'gamma': [0.5, 1,2,3,4]}]
clf = GridSearchCV(SVC(kernel='rbf'), tuned_parameters, cv=10, scoring='accuracy', return_train_score=True)
clf.fit(X_train, y_train)

Encontramos que la mejor configuración fue C=1 con gamma=1:

In [ ]:
clf.best_params_

Analicemos la matriz de confusión:

In [ ]:
confusion_matrix(y_test, clf.best_estimator_.predict(X_test))

Tenemos un 91.1% de accuracy sobre la clasificación con el mejor modelo encontrado:

In [ ]:
clf.best_estimator_.score(X_test, y_test)

Veamos como serían las curvas de la métrica ROC AUC

In [ ]:
svc = SVC(kernel="rbf", C=1, gamma=1)
svc.fit(X_train, y_train)

y_train_pred = svc.predict(X_train)
accuracy_score(y_train, y_train_pred)

In [ ]:
y_test_pred = svc.predict(X_test)
accuracy_score(y_test, y_test_pred)

In [ ]:
false_pos_rate, true_pos_rate, _ = roc_curve(y_train, y_train_pred)
roc_auc = auc(false_pos_rate, true_pos_rate)
false_pos_rate_test, true_pos_rate_test, _ = roc_curve(y_test, y_test_pred)
roc_auc_test = auc(false_pos_rate_test, true_pos_rate_test)

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(6,6))

ax1.plot(false_pos_rate, true_pos_rate, label='Training' % roc_auc, color='b')
ax1.plot(false_pos_rate_test, true_pos_rate_test, label='Test' % roc_auc_test, color='g')
ax1.set_title('Comparación del ROC del SVM')

for ax in fig.axes:
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([-0.05, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc="lower right")